<a href="https://colab.research.google.com/github/johannthefabulos/FP441/blob/main/Copy_of_Bioinformatics_Assignment_3_Starter.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [72]:
!pip install BioPython

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [73]:
from Bio.PDB import *
from Bio import pairwise2
from Bio.Align import *
from Bio import Align



In [91]:
pdbl = PDBList()

# download the pdb files
pdbl.retrieve_pdb_file("2K49",file_format='pdb')
pdbl.retrieve_pdb_file("2K8E",file_format='pdb')
parser = PDBParser()



Structure exists: '/content/k4/pdb2k49.ent' 
Structure exists: '/content/k8/pdb2k8e.ent' 


In [93]:
# note their location and file name on colab
# you may also download them as PDB files and indicate the path on your machine when running locally
str1 = parser.get_structure("protA","/content/k4/pdb2k49.ent")
str2 = parser.get_structure("protB","/content/k8/pdb2k8e.ent")



In [94]:
def aa_residues(chain):
        aa_only = []
        for i in chain:
            if i.get_resname() in standard_aa_names:
                aa_only.append(i)
        return aa_only

# get the sequence of amino acids as 1 letter aa codes
chain1 = CaPPBuilder()
chain2 = CaPPBuilder()

a = ""
for pp in chain1.build_peptides(str1):
	a = a+pp.get_sequence()
 
print(a)

b = ""
for pp in chain2.build_peptides(str2):
	b = b+pp.get_sequence()
 
print(b)

# use blosum62 for the alignment
matrix = substitution_matrices.load("BLOSUM62")

# YOUR CODE HERE for local alignment of a and b using "matrix" and gap opening and gap extension of -5
aligner = Align.PairwiseAligner()

aligner.substitution_matrix = matrix
aligner.open_gap_score = -5
aligner.extend_gap_score = -5
aln = aligner.align(a, b)[0]
print("aln",aln)
print("end aln")

# the code below gets the CA atom lists from both input structures

# get the first model from each structure
m1 = str1[0]
m2 = str2[0]

# get the first chain from each model
for c in m1:
	c1 = c
	break

for c in m2:
	c2 = c
	break

amino_acids1 = aa_residues(c1)
amino_acids2 = aa_residues(c2)

# get all the CA atoms in the chain
atoms1 = []
atoms2 = []

for r in amino_acids1:
	atoms1.append(r["CA"])
 
print(len(atoms1))

for r in amino_acids2:
	atoms2.append(r["CA"])

print(len(atoms2))

# now construct the list of matched atoms using the local alignment
# YOUR CODE HERE
first, second = aln
score = aln.score
print("first",first)
print("first", second)
print("type",type(first))
assert (len(first) == len(second))


matoms1 = []
matoms2 = []
i = 0
j = 0
k = 0
while i < len(first):
	if first[i] != "-" and second[i] != "-":
		matoms1.append(atoms1[j])
		matoms2.append(atoms2[k])
	if first[i] != "-":
		j += 1
	if second[i] != "-":
		k += 1
	i += 1


# the code below does not use the local alignment result and
# just matches the first 4 atoms of each structure
# to demonstrate the use of superimpose
#matoms1.append(atoms1[0])
#matoms1.append(atoms1[1])
#matoms1.append(atoms1[2])
#matoms1.append(atoms1[3])

#matoms2.append(atoms2[5])
#matoms2.append(atoms2[90])
#matoms2.append(atoms2[95])
#matoms2.append(atoms2[100])

# now superimpose the matched atoms
# note that matoms1 and matoms2 are same length coordinate lists
sup = Superimposer()
sup.set_atoms(matoms1,matoms2)
print("RMSD = " + str(sup.rms))

MSGWYELSKSSNDQFKFVLKAGNGEVILTSELYTGKSGAMNGIESVQTNSPIEARYAKEVAKNDKPYFNLKAANHQIIGTSQMYSSTAARDNGIKSVMENGKTTTIKDLTLEHHHHHH
GVIMAGWFELSKSSDNQFRFVLKAGNGETILTSELYTSKTSAEKGIASVRSNSPQEERYEKKTASNGKFYFNLKAANHQIIGSSQMYATAQSRETGIASVKANGTSQTVKDNTGSN
aln target            0 ---MSGWYELSKSSNDQFKFVLKAGNGEVILTSELYTGKSGAMNGIESVQTNSPIEARYA
                  0 ---|.||.||||||..||.|||||||||.||||||||.|..|..||.||..|||.|.||.
query             0 GVIMAGWFELSKSSDNQFRFVLKAGNGETILTSELYTSKTSAEKGIASVRSNSPQEERYE

target           57 KEVAKNDKPYFNLKAANHQIIGTSQMYSSTAARDNGIKSVMENGKTTTIKDLTLEHHHHH
                 60 |..|.|.|.|||||||||||||.||||.....|..||.||..||...|.||.|-...---
query            60 KKTASNGKFYFNLKAANHQIIGSSQMYATAQSRETGIASVKANGTSQTVKDNT-GSN---

target          117 H 118
                120 - 121
query           116 - 116

end aln
118
116
first ---MSGWYELSKSSNDQFKFVLKAGNGEVILTSELYTGKSGAMNGIESVQTNSPIEARYAKEVAKNDKPYFNLKAANHQIIGTSQMYSSTAARDNGIKSVMENGKTTTIKDLTLEHHHHHH
first GVIMAGWFELSKSSDNQFRFVLKAGNGETILTSELYTSKTSAE